In [7]:
import numpy as np
import cv2
from collections import deque

# Define the upper and lower boundaries for a color to be considered "Blue"
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])

# Define a 5x5 kernel for erosion and dilation
kernel = np.ones((5, 5), np.uint8)

# Setup deques to store separate colors in separate arrays
bpoints = [deque(maxlen=512)]
gpoints = [deque(maxlen=512)]
rpoints = [deque(maxlen=512)]
ypoints = [deque(maxlen=512)]
epoints = [deque(maxlen=512)]

bindex = 0
gindex = 0
rindex = 0
yindex = 0
eindex = 0

colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255), (255,255,255)]
colorIndex = 0

# Setup the Paint interface
paintWindow = np.zeros((471,656,3)) + 255 #paint window
paintWindow = cv2.rectangle(paintWindow, (20,1), (100,65), (0,0,0), -1)
paintWindow = cv2.rectangle(paintWindow, (120,1), (195,65), colors[0], -1)
paintWindow = cv2.rectangle(paintWindow, (225,1), (310,65), colors[1], -1)
paintWindow = cv2.rectangle(paintWindow, (340,1), (425,65), colors[2], -1)
paintWindow = cv2.rectangle(paintWindow, (455,1), (540,65), colors[3], -1)
paintWindow = cv2.rectangle(paintWindow, (565,1), (645,65), (128,0,0), -1)
cv2.putText(paintWindow, "CLEAR", (35, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "BLUE", (135, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "GREEN", (238, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "RED", (370, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "YELLOW", (470, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "ERASER", (570, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)
cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)

# Load the video
camera = cv2.VideoCapture(1)

# Keep looping
while True:
    # Grab the current paintWindow
    (grabbed, frame) = camera.read()
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Add the coloring options to the frame
    frame = cv2.rectangle(frame, (20,1), (100,65), (122,122,122), -1)
    frame = cv2.rectangle(frame, (120,1), (195,65), colors[0], -1)
    frame = cv2.rectangle(frame, (225,1), (310,65), colors[1], -1)
    frame = cv2.rectangle(frame, (340,1), (425,65), colors[2], -1)
    frame = cv2.rectangle(frame, (455,1), (540,65), colors[3], -1)
    frame = cv2.rectangle(frame, (565,1), (645,65), (128,0,0), -1)
    cv2.putText(frame, "CLEAR", (35, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (145, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (248, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (370, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (470, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)
    cv2.putText(frame, "ERASER", (570, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)
    # Check to see if we have reached the end of the video
    if not grabbed:
        break

    # Determine which pixels fall within the blue boundaries and then blur the binary image
    blueMask = cv2.inRange(hsv, blueLower, blueUpper)
    blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    blueMask = cv2.dilate(blueMask, kernel, iterations=1)

    # Find contours in the image
    (_, cnts, _) = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,
    	cv2.CHAIN_APPROX_SIMPLE)
    center = None

    # Check to see if any contours were found
    if len(cnts) > 0:
    	# Sort the contours and find the largest one -- we
    	# will assume this contour correspondes to the area of the bottle cap
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        # Get the radius of the enclosing circle around the found contour
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        # Draw the circle around the contour
        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        # Get the moments to calculate the center of the contour (in this case Circle)
        M = cv2.moments(cnt)
        
        #finding the center of the circle
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

        if center[1] <= 65:
            if 20 <= center[0] <= 100: # Clear All
                bpoints = [deque(maxlen=512)]
                gpoints = [deque(maxlen=512)]
                rpoints = [deque(maxlen=512)]
                ypoints = [deque(maxlen=512)]
                epoints = [deque(maxlen=512)]
                bindex = 0
                gindex = 0
                rindex = 0
                yindex = 0
                eindex = 0
                paintWindow[67:,:,:] = 255
            elif 120 <= center[0] <= 195:
                    colorIndex = 0 # Blue
            elif 225 <= center[0] <= 310:
                    colorIndex = 1 # Green
            elif 340 <= center[0] <= 425:
                    colorIndex = 2 # Red
            elif 455 <= center[0] <= 540:
                    colorIndex = 3 # Yellow
            elif 565 <= center[0] <= 645: #eraser
                    colorIndex = 4
        else :
            if colorIndex == 0:
                bpoints[bindex].appendleft(center)
            elif colorIndex == 1:
                gpoints[gindex].appendleft(center)
            elif colorIndex == 2:
                rpoints[rindex].appendleft(center)
            elif colorIndex == 3:
                ypoints[yindex].appendleft(center)
            elif colorIndex ==4:
                epoints[eindex].appendleft(center)
    # Append the next deque when no contours are detected (i.e., bottle cap reversed,blue not detected)
    else:
        bpoints.append(deque(maxlen=512))
        bindex += 1
        gpoints.append(deque(maxlen=512))
        gindex += 1
        rpoints.append(deque(maxlen=512))
        rindex += 1
        ypoints.append(deque(maxlen=512))
        yindex += 1
        epoints.append(deque(maxlen=512))
        eindex += 1

    # Draw lines of all the colors (Blue, Green, Red and Yellow)
    points = [bpoints, gpoints, rpoints, ypoints, epoints]
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                if i==4:
                        cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 15)
                        continue
                cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                

    # Show the frame and the paintWindow image
    cv2.imshow("Tracking_frame", frame)
    cv2.imshow("PaintWindow", paintWindow)

	# If the 'q' key is pressed, stop the loop
    if cv2.waitKey(5) & 0xFF == 27:
        break

# Cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()


In [ ]:
%config IPCompleter.greedy=True
%config IPCompleter.greedy=True